https://www.kaggle.com/rtatman/forum-post-embeddings-clustering  
https://www.kaggle.com/rtatman/yake-helper-funcs

In [ ]:
import os
os.system("pip install git+https://github.com/LIAAD/yake")

In [ ]:
import os
path = "C:/Users/LIUM3478/OneDrive Corp/OneDrive - Atkins Ltd/Work_Atkins/nlp"
os.chdir(path)

import re
import pandas as pd
import numpy as np
import math

import fasttext

import Import_and_clean_data as ic
import WCS_Clustering_MainFunction as cmf
import SlimWeightedVecData as swd
import OutputWarranty as ow
import WarrantyCluster as wc
import WordFreqInClusters as wfc

import yake_helper_funcs as yhf   # custom function
from datetime import datetime, timedelta
from math import sqrt, floor
from sklearn.cluster import SpectralClustering

import itertools
from matplotlib import pyplot as plt
from flashtext.keyword import KeywordProcessor
import string
import nltk


In [ ]:
#!pip install flashtext

In [ ]:
# showing multiple outputs in one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
df_path = "C:/Users/LIUM3478/OneDrive Corp/OneDrive - Atkins Ltd/Work_Atkins/otu sap wcs 2020 12/data/df_ny_gl_system_1.csv"
stopword_path = "C:/Users/LIUM3478/OneDrive Corp/OneDrive - Atkins Ltd/Work_Atkins/nlp/danska-stopwords.csv"
fasttext_link = "cc.da.300.bin"

In [ ]:
# here Text-variable is created for analysis 
# there are some rows without any system assigned
df = ic.ImportAndCleanCSV(df_path, datenc = "ISO 8859-1", text_multi_var = True, text_var = "Beskrivelse")
df.shape
df.head()

## Obtaining data for clustering

The function DataForClustering is applied to the subset of the data (in the following example, the subset of the data is the one which system = 1.   

The output of the function returns weighted sentence vectors, and index created for each row.

In [ ]:
df_nlp = swd.DataForClustering(dat = df, group = True, 
                                      sort_var_1 = 'System', sort_var_2 = 'Meddelelsesdato',
                                      textcol = 'Text',
                                      stopwordpath = stopword_path, stopenc = "ISO 8859-1",
                                      fasttext_link = fasttext_link)

In [ ]:
pd.set_option('display.max_columns', None)  
df_nlp.head()

In [ ]:
#df_nlp.to_csv('C:/Users/LIUM3478/OneDrive Corp/OneDrive - Atkins Ltd/Work_Atkins/otu sap wcs 2020 12/processed_df.csv', 
#              sep=';', encoding='ISO 8859-1')

In [ ]:
# extact keywords & tokenize
#keywords = yhf.keywords_yake(sample_posts, )
keywords_tokenized = yhf.tokenizing_after_YAKE(df_nlp['Text'])
#keywords_tokenized
keyword_sets = [set(row) for row in keywords_tokenized]
len(keyword_sets)

In [ ]:
keywords_embedding = wsv.WeightedSentenceVector(dat = df_nlp,
                                               textcol = textcol,
                                               stopwordpath = stopwordpath,
                                               stopenc = stopenc,
                                               fasttextmodel = fasttext_link)

## Clustering

In [ ]:
## Hyperprameters

# number of clusters currently based on the square root of the # of rows in the data
# number of posts
num_of_posts = df_nlp_system.shape[0]
num_of_posts

# Number of clusters is square root of the # of posts (rounded down)
number_clusters = math.floor(math.sqrt(num_of_posts))
number_clusters

### Spectral Clustering
What does mean Spectral Clustering and where is it used:
https://www.quora.com/What-are-the-advantages-of-spectral-clustering-over-k-means-clustering
Spectral clustering: graph clustering

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html

In [ ]:
df_nlp_system.WeightVec.shape

In [ ]:
# converting sequence to array
import itertools
weightVec_array = np.array(list(itertools.zip_longest(*df_nlp_system.WeightVec, fillvalue=0))).T
weightVec_array.shape

In [ ]:
# the default k-means label assignment didn't work well
from sklearn.cluster import SpectralClustering
clustering = SpectralClustering(n_clusters=number_clusters, 
                                assign_labels="discretize",
                                n_neighbors=number_clusters).fit(weightVec_array)
clustering

In [ ]:
# generating HTML report of clusters

# get sample post info by #
def get_post_info_by_cluster(number, 
                             data,
                             cluster):
    return(data[cluster.labels_ == number])

df_nlp_system_sub = df_nlp_system.drop('WeightVec', 1)   # where 1 is the axis number (0 for rows and 1 for columns.)

for i in range(number_clusters):
    
    print(f"Cluster {i}:\n")
    print(get_post_info_by_cluster(i, 
                                   data = df_nlp_system_sub,
                                   cluster = clustering))
    print("\n")

In [ ]:
df_nlp_system_sub['Cluster'] = clustering.labels_
set(clustering.labels_.tolist())

In [ ]:
df_nlp_system_sub['Cluster'] = clustering.labels_
cluster_spectral = wfc.WordFreqInClusters(df = df_nlp_system_sub)  # num_top = 5 for frequency
cluster_spectral

### Generate HTML report of clusters

In [ ]:
# count of posts/cluster
cluster_counts = pd.Series(clustering.labels_).value_counts()
cluster_counts
# look at distrobution of cluster labels
size_df = pd.Series(clustering.labels_).value_counts().to_frame()

size_df = size_df.rename(columns={0: "size"})

#size_df['characteristic_words'] = 0
#size_df['characteristic_words'] = size_df['characteristic_words'].astype(object)

size_df["Cluster"] = size_df.index

size_df['link_to_posts'] = ""

for index, row in size_df.iterrows():
    current_cluster_label = row["Cluster"]
    link_to_posts = (f'<a href="#anchor_{current_cluster_label}">Link to posts</a>')
    size_df.at[index,'link_to_posts'] = link_to_posts

size_df = size_df.drop(index)
size_df

In [ ]:
cluster_spectral

In [ ]:
size_df = pd.merge(size_df, cluster_spectral, on='Cluster', how='left')
#size_df = size_df.join(cluster_spectral)
size_df

In [ ]:
with open("C:/Users/LIUM3478/OneDrive Corp/OneDrive - Atkins Ltd/Work_Atkins/otu sap wcs 2020 12/nlp/output/spectral_cluster_report.html", 'w', encoding="ISO 8859-1") as file:
    # file header
    file.writelines('<meta charset="ISO 8859-1">\n')
    
    # add cluster info
    file.write(size_df.drop(["Cluster"], axis=1).to_html(escape=False))
    file.write("\n")
    for i in range(number_clusters):
        if i in cluster_spectral.index:
            file.write(f'\n<h2 id="anchor_{i}">Cluster {i}:</h2>\n')
            cluster_info = get_post_info_by_cluster(i,
                                                    data = df_nlp_system_sub,
                                                    cluster = clustering)
            file.write(cluster_info.to_html(escape=False))

## K-means

### Finding Optimal Clusters
Clustering is an unsupervised operation, and KMeans requires that we specify the number of clusters. 

One simple approach is to plot the SSE for a range of cluster sizes. We look for the "elbow" where the SSE begins to level off. MiniBatchKMeans introduces some noise so I raised the batch and init sizes higher. Unfortunately the regular Kmeans implementation is too slow. You'll notice different random states will generate different charts. Here I chose 10 clusters.

In [ ]:
def find_optimal_clusters(data, max_k):
    iters = range(2, max_k+1, 2)
    
    sse = []
    for k in iters:
        sse.append(MiniBatchKMeans(n_clusters=k, init_size=1024, batch_size=2048, random_state=20).fit(data).inertia_)
        print('Fit {} clusters'.format(k))
        
    f, ax = plt.subplots(1, 1)
    ax.plot(iters, sse, marker='o')
    ax.set_xlabel('Cluster Centers')
    ax.set_xticks(iters)
    ax.set_xticklabels(iters)
    ax.set_ylabel('SSE')
    ax.set_title('SSE by Cluster Center Plot')
    
find_optimal_clusters(weightVec_array, 20)

In [ ]:
assigned_clusters = MiniBatchKMeans(n_clusters=10, 
                                    compute_labels=True,
                                    init_size=1024, batch_size=2048, random_state=20).fit(weightVec_array)
assigned_clusters.labels_

# get sample post info by #
def get_post_info_by_cluster(number, 
                             data,
                             cluster):
    return(data[assigned_clusters.labels_ == number])

df_nlp_system_sub = df_nlp_system.drop('WeightVec', 1)   # where 1 is the axis number (0 for rows and 1 for columns.)

for i in range(11):
    
    print(f"Cluster {i}:\n")
    print(get_post_info_by_cluster(i, 
                                   data = df_nlp_system_sub,
                                   cluster = assigned_clusters))
    print("\n")

In [ ]:
# count of posts/cluster
pd.Series(assigned_clusters.labels_).value_counts()

In [ ]:
i = 0
print(f"Cluster {i}:\n")
print(get_post_info_by_cluster(i, 
                                   data = df_nlp_system_sub,
                                   cluster = assigned_clusters))


In [ ]:
kmeans_cluster_df = get_post_info_by_cluster(i, 
                                             data = df_nlp_system_sub,
                                             cluster = assigned_clusters)
# kmeans_cluster_df.to_csv('C:/Users/LIUM3478/OneDrive Corp/OneDrive - Atkins Ltd/Work_Atkins/otu sap wcs 2020 12/nlp/output/kmeans_cluster.csv', 
#               sep=';', encoding='ISO 8859-1')
kmeans_cluster_df

In [ ]:
# look at distrobution of cluster labels
size_df = pd.Series(assigned_clusters.labels_).value_counts().to_frame()
size_df = size_df.rename(columns={0: "size"})
size_df['characteristic_words'] = 0
size_df['characteristic_words'] = size_df['characteristic_words'].astype(object)
size_df["cluster_label"] = size_df.index
size_df['link_to_posts'] = ""

for index, row in size_df.iterrows():
    current_cluster_label = row["cluster_label"]
    link_to_posts = (f'<a href="#anchor_{current_cluster_label}">Link to posts</a>')
    size_df.at[index,'link_to_posts'] = link_to_posts
    
size_df

In [ ]:
df_nlp_system_sub['Cluster'] = assigned_clusters.labels_
cluster_kmeans = wfc.WordFreqInClusters(df = df_nlp_system_sub)  # num_top = 5 for frequency
cluster_kmeans

In [ ]:
size_df = size_df.join(cluster_kmeans)
size_df

In [ ]:
# write and save cluster information to a html-file
with open("C:/Users/LIUM3478/OneDrive Corp/OneDrive - Atkins Ltd/Work_Atkins/otu sap wcs 2020 12/nlp/output/cluster_report.html", 'w', encoding="ISO 8859-1") as file:
    # file header
    file.writelines('<meta charset="ISO 8859-1">\n')
    
    # add cluster info
    file.write(size_df.drop(["cluster_label"], axis=1).to_html(escape=False))
    file.write("\n")
    for i in range(10):
        if i in cluster_kmeans.index:
            file.write(f'\n<h2 id="anchor_{i}">Cluster {i}:</h2>\n')
            cluster_info = get_post_info_by_cluster(i,
                                                    data = df_nlp_system_sub,
                                                    cluster = clustering)
            file.write(cluster_info.to_html(escape=False))


In [ ]:
assigned_clusters = ClusterWordVec(w, num_clusts = 10, n_init = 20, random_state = 100)

In [ ]:
df_1['Cluster'] = assigned_clusters
df_1.groupby('Cluster').first()

In [ ]:
cluster_wordfreq = WordFreqInClusters(df = df_1, num_top = 5)

In [ ]:
print(cluster_wordfreq)

In [ ]:
test = df_1[df_1['Cluster'].values == 1]
test['System'].unique()
test